Convolutiona Deep Network for Image Classifica
=============

The goal of this project is to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters usingconvolutional neural network.

In [30]:
#import all packages
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [31]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f, encoding='latin1')
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [32]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [33]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/ predictions.shape[0])

---
ConvNet
---------

Setting up the computational graph and tuning the parameters to maximize accuracy.

---

In [46]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 128

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))

    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_hidden], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer5_weights = tf.Variable(tf.truncated_normal([num_hidden, num_hidden], stddev=0.1))
    layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer6_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer6_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    global_step = tf.Variable(0)
  
    # Model.
    def model(data):
        conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        hidden1 = tf.nn.relu(conv1 + layer1_biases)
        
        conv2 = tf.nn.conv2d(hidden1, layer2_weights, [1, 1, 1, 1], padding='SAME')
        conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        hidden2 = tf.nn.relu(conv2 + layer2_biases)
        
        
        shape = hidden2.get_shape().as_list()
        hidden2 = tf.reshape(hidden2, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden3 = tf.nn.relu(tf.matmul(hidden2, layer3_weights) + layer3_biases)
        hidden4 = tf.nn.relu(tf.matmul(hidden3, layer4_weights) + layer4_biases)
        hidden5 = tf.nn.relu(tf.matmul(hidden4, layer5_weights) + layer5_biases)
        return hidden5
  
    # Training computation.
    hidden = model(tf_train_dataset)
    layer6_weights_drop = tf.nn.dropout(layer6_weights, 0.5)
    logits = tf.matmul(hidden, layer6_weights_drop) + layer6_biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    learning_rate = tf.train.exponential_decay(0.05, global_step, 100000, 0.96)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_hidden = model(tf_valid_dataset)
    valid_logits = logits = tf.matmul(valid_hidden, layer6_weights) + layer6_biases
    valid_prediction = tf.nn.softmax(valid_logits)
    
    test_hidden = model(tf_test_dataset)
    test_logits = logits = tf.matmul(test_hidden, layer6_weights) + layer6_biases
    test_prediction = tf.nn.softmax(test_logits)

In [47]:
num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.904014
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.846807
Minibatch accuracy: 37.5%
Validation accuracy: 36.9%
Minibatch loss at step 100: 1.657497
Minibatch accuracy: 31.2%
Validation accuracy: 57.5%
Minibatch loss at step 150: 0.899516
Minibatch accuracy: 75.0%
Validation accuracy: 67.8%
Minibatch loss at step 200: 1.297317
Minibatch accuracy: 75.0%
Validation accuracy: 73.1%
Minibatch loss at step 250: 1.410429
Minibatch accuracy: 50.0%
Validation accuracy: 73.0%
Minibatch loss at step 300: 0.613570
Minibatch accuracy: 75.0%
Validation accuracy: 75.5%
Minibatch loss at step 350: 0.678321
Minibatch accuracy: 93.8%
Validation accuracy: 78.0%
Minibatch loss at step 400: 0.315147
Minibatch accuracy: 100.0%
Validation accuracy: 80.7%
Minibatch loss at step 450: 0.816128
Minibatch accuracy: 75.0%
Validation accuracy: 79.2%
Minibatch loss at step 500: 0.794324
Minibatch accuracy: 81.2%
Validation accuracy: 80.2%


Testing accuracy using avg_pool instead of max_pool

In [44]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 128

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))

    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_hidden], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer5_weights = tf.Variable(tf.truncated_normal([num_hidden, num_hidden], stddev=0.1))
    layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer6_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer6_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    global_step = tf.Variable(0)
  
    # Model.
    def model(data):
        conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        conv1 = tf.nn.avg_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        hidden1 = tf.nn.relu(conv1 + layer1_biases)
        
        conv2 = tf.nn.conv2d(hidden1, layer2_weights, [1, 1, 1, 1], padding='SAME')
        conv2 = tf.nn.avg_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        hidden2 = tf.nn.relu(conv2 + layer2_biases)
        
        
        shape = hidden2.get_shape().as_list()
        hidden2 = tf.reshape(hidden2, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden3 = tf.nn.relu(tf.matmul(hidden2, layer3_weights) + layer3_biases)
        hidden4 = tf.nn.relu(tf.matmul(hidden3, layer4_weights) + layer4_biases)
        hidden5 = tf.nn.relu(tf.matmul(hidden4, layer5_weights) + layer5_biases)
        return hidden5
  
    # Training computation.
    hidden = model(tf_train_dataset)
    layer6_weights_drop = tf.nn.dropout(layer6_weights, 0.5)
    logits = tf.matmul(hidden, layer6_weights_drop) + layer6_biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    learning_rate = tf.train.exponential_decay(0.05, global_step, 100000, 0.96)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_hidden = model(tf_valid_dataset)
    valid_logits = logits = tf.matmul(valid_hidden, layer6_weights) + layer6_biases
    valid_prediction = tf.nn.softmax(valid_logits)
    
    test_hidden = model(tf_test_dataset)
    test_logits = logits = tf.matmul(test_hidden, layer6_weights) + layer6_biases
    test_prediction = tf.nn.softmax(test_logits)

In [45]:
num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.567846
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.106023
Minibatch accuracy: 12.5%
Validation accuracy: 28.0%
Minibatch loss at step 100: 1.288450
Minibatch accuracy: 50.0%
Validation accuracy: 63.5%
Minibatch loss at step 150: 0.965131
Minibatch accuracy: 56.2%
Validation accuracy: 66.4%
Minibatch loss at step 200: 1.014105
Minibatch accuracy: 68.8%
Validation accuracy: 74.8%
Minibatch loss at step 250: 1.504605
Minibatch accuracy: 68.8%
Validation accuracy: 75.0%
Minibatch loss at step 300: 0.554283
Minibatch accuracy: 81.2%
Validation accuracy: 77.2%
Minibatch loss at step 350: 0.489405
Minibatch accuracy: 87.5%
Validation accuracy: 77.5%
Minibatch loss at step 400: 0.694357
Minibatch accuracy: 87.5%
Validation accuracy: 79.6%
Minibatch loss at step 450: 0.905295
Minibatch accuracy: 75.0%
Validation accuracy: 79.4%
Minibatch loss at step 500: 1.014445
Minibatch accuracy: 81.2%
Validation accuracy: 80.0%
Mi

Checking the effect of local_response_normalization after pooling of the second Conv layer

In [51]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))

    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_hidden], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer5_weights = tf.Variable(tf.truncated_normal([num_hidden, num_hidden], stddev=0.1))
    layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer6_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer6_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    global_step = tf.Variable(0)
  
    # Model.
    def model(data):
        conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        hidden1 = tf.nn.relu(conv1 + layer1_biases)
        
        conv2 = tf.nn.conv2d(hidden1, layer2_weights, [1, 1, 1, 1], padding='SAME')
        conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        norm1 = tf.nn.local_response_normalization(conv2, 4, bias=1.0,\
                                                         alpha=0.001 / 9.0, beta=0.75)
        
        hidden2 = tf.nn.relu(norm1 + layer2_biases)
        
        
        shape = hidden2.get_shape().as_list()
        hidden2 = tf.reshape(hidden2, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden3 = tf.nn.relu(tf.matmul(hidden2, layer3_weights) + layer3_biases)
        hidden4 = tf.nn.relu(tf.matmul(hidden3, layer4_weights) + layer4_biases)
        hidden5 = tf.nn.relu(tf.matmul(hidden4, layer5_weights) + layer5_biases)
        return hidden5
  
    # Training computation.
    hidden = model(tf_train_dataset)
    layer6_weights_drop = tf.nn.dropout(layer6_weights, 0.5)
    logits = tf.matmul(hidden, layer6_weights_drop) + layer6_biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    learning_rate = tf.train.exponential_decay(0.05, global_step, 100000, 0.96)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_hidden = model(tf_valid_dataset)
    valid_logits = logits = tf.matmul(valid_hidden, layer6_weights) + layer6_biases
    valid_prediction = tf.nn.softmax(valid_logits)
    
    test_hidden = model(tf_test_dataset)
    test_logits = logits = tf.matmul(test_hidden, layer6_weights) + layer6_biases
    test_prediction = tf.nn.softmax(test_logits)

In [52]:
num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.324430
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.296969
Minibatch accuracy: 18.8%
Validation accuracy: 13.1%
Minibatch loss at step 100: 2.298706
Minibatch accuracy: 6.2%
Validation accuracy: 10.7%
Minibatch loss at step 150: 2.297452
Minibatch accuracy: 6.2%
Validation accuracy: 10.1%
Minibatch loss at step 200: 2.286797
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 250: 2.358565
Minibatch accuracy: 0.0%
Validation accuracy: 13.9%
Minibatch loss at step 300: 2.199790
Minibatch accuracy: 6.2%
Validation accuracy: 26.9%
Minibatch loss at step 350: 2.084693
Minibatch accuracy: 6.2%
Validation accuracy: 38.8%
Minibatch loss at step 400: 1.452913
Minibatch accuracy: 43.8%
Validation accuracy: 50.0%
Minibatch loss at step 450: 1.333013
Minibatch accuracy: 43.8%
Validation accuracy: 56.9%
Minibatch loss at step 500: 1.614483
Minibatch accuracy: 43.8%
Validation accuracy: 61.3%
Minibat

In [65]:
batch_size = 16
patch_size = 5
depth = 10
num_hidden = 128

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.zeros([depth]))
    
    layer3_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[depth]))

    layer4_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer5_weights = tf.Variable(tf.truncated_normal([num_hidden, num_hidden], stddev=0.1))
    layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer6_weights = tf.Variable(tf.truncated_normal([num_hidden, num_hidden], stddev=0.1))
    layer6_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer7_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer7_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    global_step = tf.Variable(0)
  
    # Model.
    def model(data):
        conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden1 = tf.nn.relu(conv1 + layer1_biases)
        
        conv2 = tf.nn.conv2d(conv1, layer2_weights, [1, 1, 1, 1], padding='SAME')
        conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        hidden2 = tf.nn.relu(conv2 + layer2_biases)
        
        conv3 = tf.nn.conv2d(hidden2, layer3_weights, [1, 1, 1, 1], padding='SAME')
        conv3 = tf.nn.max_pool(conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        hidden3 = tf.nn.relu(conv3 + layer3_biases)
        
        
        shape = hidden3.get_shape().as_list()
        hidden3 = tf.reshape(hidden3, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden4 = tf.nn.relu(tf.matmul(hidden3, layer4_weights) + layer4_biases)
        hidden5 = tf.nn.relu(tf.matmul(hidden4, layer5_weights) + layer5_biases)
        hidden6 = tf.nn.relu(tf.matmul(hidden5, layer6_weights) + layer6_biases)
        return hidden6
  
    # Training computation.
    hidden = model(tf_train_dataset)
    layer7_weights_drop = tf.nn.dropout(layer7_weights, 0.5)
    logits = tf.matmul(hidden, layer7_weights_drop) + layer7_biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    learning_rate = tf.train.exponential_decay(0.05, global_step, 100000, 0.96)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_hidden = model(tf_valid_dataset)
    valid_logits = logits = tf.matmul(valid_hidden, layer7_weights) + layer7_biases
    valid_prediction = tf.nn.softmax(valid_logits)
    
    test_hidden = model(tf_test_dataset)
    test_logits = logits = tf.matmul(test_hidden, layer7_weights) + layer7_biases
    test_prediction = tf.nn.softmax(test_logits)

In [66]:
num_steps = 20001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.197231
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.887174
Minibatch accuracy: 37.5%
Validation accuracy: 41.5%
Minibatch loss at step 100: 1.788808
Minibatch accuracy: 43.8%
Validation accuracy: 51.0%
Minibatch loss at step 150: 2.031487
Minibatch accuracy: 31.2%
Validation accuracy: 54.2%
Minibatch loss at step 200: 0.949892
Minibatch accuracy: 81.2%
Validation accuracy: 70.0%
Minibatch loss at step 250: 1.490751
Minibatch accuracy: 43.8%
Validation accuracy: 74.6%
Minibatch loss at step 300: 0.499875
Minibatch accuracy: 81.2%
Validation accuracy: 78.2%
Minibatch loss at step 350: 0.584407
Minibatch accuracy: 81.2%
Validation accuracy: 77.2%
Minibatch loss at step 400: 0.542519
Minibatch accuracy: 81.2%
Validation accuracy: 78.6%
Minibatch loss at step 450: 1.041101
Minibatch accuracy: 81.2%
Validation accuracy: 78.4%
Minibatch loss at step 500: 0.623874
Minibatch accuracy: 81.2%
Validation accuracy: 80.8%
M